# Rule-Based Remediation

## Overview
This notebook implements deterministic rule-based remediation logic that maps detected anomalies to specific healing actions. Rules are executed through the coordination engine.

## Prerequisites
- Completed: All Phase 2 notebooks (anomaly detection)
- Coordination engine running
- Ensemble predictions available

## Learning Objectives
- Define remediation rules
- Map anomalies to actions
- Execute kubectl commands
- Validate remediation success
- Track remediation outcomes

## Key Concepts
- **Rule Engine**: Deterministic decision logic
- **Action Mapping**: Anomaly type → Remediation action
- **Kubectl Integration**: Execute Kubernetes operations
- **Validation**: Verify remediation effectiveness

## Setup Section

In [ ]:
import sys
import os
import json
import logging
import requests
from pathlib import Path
from datetime import datetime
import subprocess
from enum import Enum
from typing import Dict, List, Optional, Any
from dataclasses import dataclass

# Setup path for utils module
def find_utils_path():
    """Find utils path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent / 'utils' if '__file__' in dir() else None,
        Path.cwd() / 'notebooks' / 'utils',
        Path.cwd().parent / 'utils',
        Path('/workspace/repo/notebooks/utils'),
        Path('/opt/app-root/src/notebooks/utils'),
    ]
    for p in possible_paths:
        if p and p.exists() and (p / 'common_functions.py').exists():
            return str(p)
    return None

utils_path = find_utils_path()
if utils_path:
    sys.path.insert(0, utils_path)
    print(f"✅ Utils path found: {utils_path}")

# Try to import common functions
try:
    from common_functions import setup_environment
    print("✅ Common functions imported")
except ImportError:
    def setup_environment():
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        os.makedirs('/opt/app-root/src/models', exist_ok=True)
        return {'data_dir': '/opt/app-root/src/data', 'models_dir': '/opt/app-root/src/models'}

# Import coordination engine client
try:
    from coordination_engine_client import get_client
    print("✅ Coordination engine client imported")
except ImportError:
    print("⚠️ Coordination engine client not available")
    get_client = None

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Setup environment
env_info = setup_environment()
logger.info(f"Environment ready: {env_info}")

# Define paths
DATA_DIR = Path('/opt/app-root/src/data')
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR = Path('/opt/app-root/src/models')

logger.info("✅ Setup complete")

    

## Implementation Section

### 1. Define Remediation Rules

In [ ]:
# Define remediation rules
REMEDIATION_RULES = {
    'high_cpu': {
        'description': 'High CPU usage detected',
        'actions': [
            {'type': 'scale', 'target': 'deployment', 'replicas': 3},
            {'type': 'restart', 'target': 'pod'}
        ]
    },
    'high_memory': {
        'description': 'High memory usage detected',
        'actions': [
            {'type': 'scale', 'target': 'deployment', 'replicas': 2},
            {'type': 'restart', 'target': 'pod'}
        ]
    },
    'pod_crash': {
        'description': 'Pod crash loop detected',
        'actions': [
            {'type': 'restart', 'target': 'pod'},
            {'type': 'check_logs', 'target': 'pod'}
        ]
    },
    'network_issue': {
        'description': 'Network connectivity issue',
        'actions': [
            {'type': 'restart', 'target': 'pod'},
            {'type': 'check_network', 'target': 'node'}
        ]
    }
}

logger.info(f"Loaded {len(REMEDIATION_RULES)} remediation rules")
for rule_name, rule_config in REMEDIATION_RULES.items():
    logger.info(f"  - {rule_name}: {rule_config['description']}")

    TARGET_METRICS = [
    # Resource Metrics (5)
    'node_memory_utilization',      # Node memory usage %
    'pod_cpu_usage',                # Pod CPU cores
    'pod_memory_usage',             # Pod memory bytes
    'alt_cpu_usage',                # Container CPU rate
    'alt_memory_usage',             # Container RSS memory
    
    # Stability Metrics (3)
    'container_restart_count',      # Total restarts
    'container_restart_rate_1h',    # Restart velocity
    'deployment_unavailable',       # Unavailable replicas
    
    # Pod Status Metrics (4)
    'namespace_pod_count',          # Pods per namespace
    'pods_pending',                 # Scheduling issues
    'pods_running',                 # Healthy pods
    'pods_failed',                  # Failed pods
    
    # Storage Metrics (2)
    'persistent_volume_usage',      # PVC usage %
    'cluster_resource_quota',       # Quota usage
    
    # Control Plane Metrics (2)
    'apiserver_request_total',      # API request rate
    'apiserver_error_rate',         # API error %
]

# Metric categories for grouping
class MetricCategory(Enum):
    RESOURCE = "resource"
    STABILITY = "stability"
    POD_STATUS = "pod_status"
    STORAGE = "storage"
    CONTROL_PLANE = "control_plane"

METRIC_CATEGORIES = {
    'node_memory_utilization': MetricCategory.RESOURCE,
    'pod_cpu_usage': MetricCategory.RESOURCE,
    'pod_memory_usage': MetricCategory.RESOURCE,
    'alt_cpu_usage': MetricCategory.RESOURCE,
    'alt_memory_usage': MetricCategory.RESOURCE,
    'container_restart_count': MetricCategory.STABILITY,
    'container_restart_rate_1h': MetricCategory.STABILITY,
    'deployment_unavailable': MetricCategory.STABILITY,
    'namespace_pod_count': MetricCategory.POD_STATUS,
    'pods_pending': MetricCategory.POD_STATUS,
    'pods_running': MetricCategory.POD_STATUS,
    'pods_failed': MetricCategory.POD_STATUS,
    'persistent_volume_usage': MetricCategory.STORAGE,
    'cluster_resource_quota': MetricCategory.STORAGE,
    'apiserver_request_total': MetricCategory.CONTROL_PLANE,
    'apiserver_error_rate': MetricCategory.CONTROL_PLANE,
}

print(f"📊 Target metrics: {len(TARGET_METRICS)}")
for category in MetricCategory:
    count = sum(1 for m, c in METRIC_CATEGORIES.items() if c == category)
    print(f"   {category.value}: {count} metrics")



In [ ]:
#Metric Threshold
METRIC_THRESHOLDS = {
    # Resource Metrics
    'node_memory_utilization': {
        'warning': 75,
        'critical': 90,
        'unit': 'percent',
        'direction': 'above'  # Anomaly when above threshold
    },
    'pod_cpu_usage': {
        'warning': 0.8,       # 800m cores
        'critical': 0.95,     # 950m cores
        'unit': 'cores',
        'direction': 'above'
    },
    'pod_memory_usage': {
        'warning': 500 * 1024 * 1024,   # 500MB
        'critical': 900 * 1024 * 1024,  # 900MB
        'unit': 'bytes',
        'direction': 'above'
    },
    'alt_cpu_usage': {
        'warning': 0.8,
        'critical': 0.95,
        'unit': 'cores',
        'direction': 'above'
    },
    'alt_memory_usage': {
        'warning': 500 * 1024 * 1024,
        'critical': 900 * 1024 * 1024,
        'unit': 'bytes',
        'direction': 'above'
    },
    
    # Stability Metrics
    'container_restart_count': {
        'warning': 3,
        'critical': 5,
        'unit': 'count',
        'direction': 'above',
        'window': '1h'
    },
    'container_restart_rate_1h': {
        'warning': 2,        # 2 restarts/hour
        'critical': 5,       # 5 restarts/hour
        'unit': 'restarts/hour',
        'direction': 'above'
    },
    'deployment_unavailable': {
        'warning': 1,
        'critical': 2,
        'unit': 'replicas',
        'direction': 'above'
    },
    
    # Pod Status Metrics
    'namespace_pod_count': {
        'warning': 100,      # Many pods might indicate issue
        'critical': 150,
        'unit': 'pods',
        'direction': 'above'
    },
    'pods_pending': {
        'warning': 3,
        'critical': 10,
        'unit': 'pods',
        'direction': 'above'
    },
    'pods_running': {
        'warning': 0,        # 0 running might be issue
        'critical': 0,
        'unit': 'pods',
        'direction': 'below'  # Anomaly when below threshold
    },
    'pods_failed': {
        'warning': 1,
        'critical': 3,
        'unit': 'pods',
        'direction': 'above'
    },
    
    # Storage Metrics
    'persistent_volume_usage': {
        'warning': 75,
        'critical': 90,
        'unit': 'percent',
        'direction': 'above'
    },
    'cluster_resource_quota': {
        'warning': 80,
        'critical': 95,
        'unit': 'percent',
        'direction': 'above'
    },
    
    # Control Plane Metrics
    'apiserver_request_total': {
        'warning': 1000,     # High request rate
        'critical': 5000,
        'unit': 'requests/sec',
        'direction': 'above'
    },
    'apiserver_error_rate': {
        'warning': 1,        # 1% error rate
        'critical': 5,       # 5% error rate
        'unit': 'percent',
        'direction': 'above'
    },
}

print(f"✅ Defined thresholds for {len(METRIC_THRESHOLDS)} metrics")


In [ ]:
# =============================================================================
# CELL 4: REMEDIATION ACTION TYPES (new cell)
# =============================================================================

class ActionType(Enum):
    # Pod Actions
    RESTART_POD = "restart_pod"
    DELETE_POD = "delete_pod"
    EVICT_POD = "evict_pod"
    
    # Scaling Actions
    SCALE_UP = "scale_up"
    SCALE_DOWN = "scale_down"
    SCALE_HPA = "scale_hpa"
    
    # Resource Actions
    INCREASE_LIMITS = "increase_limits"
    DECREASE_LIMITS = "decrease_limits"
    ADJUST_REQUESTS = "adjust_requests"
    
    # Storage Actions
    EXPAND_PVC = "expand_pvc"
    CLEANUP_STORAGE = "cleanup_storage"
    
    # Node Actions
    CORDON_NODE = "cordon_node"
    DRAIN_NODE = "drain_node"
    UNCORDON_NODE = "uncordon_node"
    
    # Diagnostic Actions
    COLLECT_LOGS = "collect_logs"
    DESCRIBE_RESOURCE = "describe_resource"
    CHECK_EVENTS = "check_events"
    
    # Alert Actions
    SEND_ALERT = "send_alert"
    CREATE_INCIDENT = "create_incident"
    
    # No Action
    MONITOR_ONLY = "monitor_only"


@dataclass
class RemediationAction:
    """Single remediation action"""
    action_type: ActionType
    target_type: str  # pod, deployment, node, pvc, etc.
    target_name: Optional[str] = None
    namespace: str = "self-healing-platform"
    parameters: Dict = None
    priority: int = 1  # 1=highest
    estimated_duration_sec: int = 30
    requires_approval: bool = False
    
    def __post_init__(self):
        if self.parameters is None:
            self.parameters = {}


# =============================================================================
# CELL 5: COMPREHENSIVE REMEDIATION RULES (replace existing rules cell)
# =============================================================================

# Map each metric to specific remediation rules
REMEDIATION_RULES = {
    # =========================================================================
    # RESOURCE METRICS (5)
    # =========================================================================
    'node_memory_utilization': {
        'description': 'High node memory utilization',
        'category': MetricCategory.RESOURCE,
        'severity_map': {
            'warning': 'medium',
            'critical': 'high'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.EVICT_POD, 'pod', parameters={'selector': 'priority=low'}),
                    RemediationAction(ActionType.SCALE_DOWN, 'deployment', parameters={'replicas_delta': -1}),
                    RemediationAction(ActionType.SEND_ALERT, 'notification', parameters={'channel': 'ops'}),
                ],
                'escalation': 'drain_node'
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.COLLECT_LOGS, 'node'),
                    RemediationAction(ActionType.CHECK_EVENTS, 'node'),
                    RemediationAction(ActionType.MONITOR_ONLY, 'metric'),
                ]
            }
        ]
    },
    
    'pod_cpu_usage': {
        'description': 'High pod CPU usage',
        'category': MetricCategory.RESOURCE,
        'severity_map': {
            'warning': 'low',
            'critical': 'high'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.SCALE_UP, 'deployment', parameters={'replicas_delta': 1}),
                    RemediationAction(ActionType.INCREASE_LIMITS, 'pod', parameters={'cpu': '200m'}),
                    RemediationAction(ActionType.RESTART_POD, 'pod', priority=2),
                ]
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.SCALE_HPA, 'hpa', parameters={'min_replicas_delta': 1}),
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod'),
                ]
            }
        ]
    },
    
    'pod_memory_usage': {
        'description': 'High pod memory usage (potential leak)',
        'category': MetricCategory.RESOURCE,
        'severity_map': {
            'warning': 'medium',
            'critical': 'high'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.RESTART_POD, 'pod', priority=1),  # Fastest fix for memory leak
                    RemediationAction(ActionType.SCALE_UP, 'deployment', parameters={'replicas_delta': 1}),
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod', parameters={'tail': 1000}),
                ]
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.INCREASE_LIMITS, 'pod', parameters={'memory': '256Mi'}),
                    RemediationAction(ActionType.MONITOR_ONLY, 'metric', parameters={'duration': '5m'}),
                ]
            }
        ]
    },
    
    'alt_cpu_usage': {
        'description': 'High container CPU rate',
        'category': MetricCategory.RESOURCE,
        'severity_map': {
            'warning': 'low',
            'critical': 'medium'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.SCALE_UP, 'deployment', parameters={'replicas_delta': 1}),
                    RemediationAction(ActionType.DESCRIBE_RESOURCE, 'pod'),
                ]
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.MONITOR_ONLY, 'metric'),
                ]
            }
        ]
    },
    
    'alt_memory_usage': {
        'description': 'High container RSS memory',
        'category': MetricCategory.RESOURCE,
        'severity_map': {
            'warning': 'medium',
            'critical': 'high'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.RESTART_POD, 'pod'),
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod'),
                ]
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.MONITOR_ONLY, 'metric'),
                ]
            }
        ]
    },
    
    # =========================================================================
    # STABILITY METRICS (3)
    # =========================================================================
    'container_restart_count': {
        'description': 'Container restart loop detected',
        'category': MetricCategory.STABILITY,
        'severity_map': {
            'warning': 'medium',
            'critical': 'critical'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod', parameters={'previous': True}),
                    RemediationAction(ActionType.CHECK_EVENTS, 'pod'),
                    RemediationAction(ActionType.DESCRIBE_RESOURCE, 'pod'),
                    RemediationAction(ActionType.CREATE_INCIDENT, 'incident', parameters={'severity': 'P2'}),
                ]
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod'),
                    RemediationAction(ActionType.CHECK_EVENTS, 'pod'),
                ]
            }
        ]
    },
    
    'container_restart_rate_1h': {
        'description': 'High restart velocity - crash loop likely',
        'category': MetricCategory.STABILITY,
        'severity_map': {
            'warning': 'high',
            'critical': 'critical'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.SCALE_DOWN, 'deployment', parameters={'replicas': 0}, requires_approval=True),
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod', parameters={'previous': True, 'all_containers': True}),
                    RemediationAction(ActionType.CREATE_INCIDENT, 'incident', parameters={'severity': 'P1'}),
                ],
                'escalation': 'page_oncall'
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.RESTART_POD, 'pod'),
                    RemediationAction(ActionType.SEND_ALERT, 'notification'),
                ]
            }
        ]
    },
    
    'deployment_unavailable': {
        'description': 'Deployment has unavailable replicas',
        'category': MetricCategory.STABILITY,
        'severity_map': {
            'warning': 'medium',
            'critical': 'high'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.SCALE_UP, 'deployment', parameters={'replicas_delta': 2}),
                    RemediationAction(ActionType.CHECK_EVENTS, 'deployment'),
                    RemediationAction(ActionType.DESCRIBE_RESOURCE, 'deployment'),
                    RemediationAction(ActionType.SEND_ALERT, 'notification', parameters={'channel': 'ops'}),
                ]
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.SCALE_UP, 'deployment', parameters={'replicas_delta': 1}),
                    RemediationAction(ActionType.CHECK_EVENTS, 'deployment'),
                ]
            }
        ]
    },
    
    # =========================================================================
    # POD STATUS METRICS (4)
    # =========================================================================
    'namespace_pod_count': {
        'description': 'Abnormal pod count in namespace',
        'category': MetricCategory.POD_STATUS,
        'severity_map': {
            'warning': 'low',
            'critical': 'medium'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.CHECK_EVENTS, 'namespace'),
                    RemediationAction(ActionType.DESCRIBE_RESOURCE, 'namespace'),
                    RemediationAction(ActionType.SEND_ALERT, 'notification'),
                ]
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.MONITOR_ONLY, 'metric'),
                ]
            }
        ]
    },
    
    'pods_pending': {
        'description': 'Pods stuck in Pending state - scheduling issue',
        'category': MetricCategory.POD_STATUS,
        'severity_map': {
            'warning': 'medium',
            'critical': 'high'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.CHECK_EVENTS, 'pod', parameters={'field_selector': 'status.phase=Pending'}),
                    RemediationAction(ActionType.DESCRIBE_RESOURCE, 'pod'),
                    RemediationAction(ActionType.UNCORDON_NODE, 'node', parameters={'selector': 'all'}),  # Try uncordoning
                    RemediationAction(ActionType.CREATE_INCIDENT, 'incident', parameters={'severity': 'P2'}),
                ]
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.CHECK_EVENTS, 'pod'),
                    RemediationAction(ActionType.DESCRIBE_RESOURCE, 'node'),
                ]
            }
        ]
    },
    
    'pods_running': {
        'description': 'No running pods - service down',
        'category': MetricCategory.POD_STATUS,
        'severity_map': {
            'warning': 'high',
            'critical': 'critical'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.SCALE_UP, 'deployment', parameters={'replicas': 1}),
                    RemediationAction(ActionType.CHECK_EVENTS, 'deployment'),
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod', parameters={'previous': True}),
                    RemediationAction(ActionType.CREATE_INCIDENT, 'incident', parameters={'severity': 'P1'}),
                ],
                'escalation': 'page_oncall'
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.CHECK_EVENTS, 'deployment'),
                    RemediationAction(ActionType.SEND_ALERT, 'notification'),
                ]
            }
        ]
    },
    
    'pods_failed': {
        'description': 'Pods in Failed state',
        'category': MetricCategory.POD_STATUS,
        'severity_map': {
            'warning': 'medium',
            'critical': 'high'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.DELETE_POD, 'pod', parameters={'field_selector': 'status.phase=Failed'}),
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod', parameters={'previous': True}),
                    RemediationAction(ActionType.CHECK_EVENTS, 'pod'),
                    RemediationAction(ActionType.SEND_ALERT, 'notification'),
                ]
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod'),
                    RemediationAction(ActionType.CHECK_EVENTS, 'pod'),
                ]
            }
        ]
    },
    
    # =========================================================================
    # STORAGE METRICS (2)
    # =========================================================================
    'persistent_volume_usage': {
        'description': 'PVC running out of space',
        'category': MetricCategory.STORAGE,
        'severity_map': {
            'warning': 'medium',
            'critical': 'high'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.EXPAND_PVC, 'pvc', parameters={'increase_percent': 50}, requires_approval=True),
                    RemediationAction(ActionType.CLEANUP_STORAGE, 'pvc', parameters={'older_than_days': 7}),
                    RemediationAction(ActionType.SEND_ALERT, 'notification', parameters={'channel': 'storage'}),
                    RemediationAction(ActionType.CREATE_INCIDENT, 'incident', parameters={'severity': 'P2'}),
                ]
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.CLEANUP_STORAGE, 'pvc', parameters={'older_than_days': 30}),
                    RemediationAction(ActionType.SEND_ALERT, 'notification'),
                ]
            }
        ]
    },
    
    'cluster_resource_quota': {
        'description': 'Resource quota nearly exhausted',
        'category': MetricCategory.STORAGE,
        'severity_map': {
            'warning': 'medium',
            'critical': 'high'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.SCALE_DOWN, 'deployment', parameters={'selector': 'priority=low', 'replicas_delta': -1}),
                    RemediationAction(ActionType.SEND_ALERT, 'notification', parameters={'channel': 'capacity'}),
                    RemediationAction(ActionType.CREATE_INCIDENT, 'incident', parameters={'type': 'capacity'}),
                ]
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.DESCRIBE_RESOURCE, 'resourcequota'),
                    RemediationAction(ActionType.SEND_ALERT, 'notification'),
                ]
            }
        ]
    },
    
    # =========================================================================
    # CONTROL PLANE METRICS (2)
    # =========================================================================
    'apiserver_request_total': {
        'description': 'API server under heavy load',
        'category': MetricCategory.CONTROL_PLANE,
        'severity_map': {
            'warning': 'medium',
            'critical': 'high'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.SEND_ALERT, 'notification', parameters={'channel': 'platform'}),
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod', parameters={'namespace': 'openshift-kube-apiserver'}),
                    RemediationAction(ActionType.CREATE_INCIDENT, 'incident', parameters={'severity': 'P1', 'type': 'platform'}),
                ],
                'escalation': 'page_platform_team'
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.MONITOR_ONLY, 'metric', parameters={'duration': '5m'}),
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod', parameters={'namespace': 'openshift-kube-apiserver', 'tail': 100}),
                ]
            }
        ]
    },
    
    'apiserver_error_rate': {
        'description': 'High API server error rate',
        'category': MetricCategory.CONTROL_PLANE,
        'severity_map': {
            'warning': 'high',
            'critical': 'critical'
        },
        'rules': [
            {
                'condition': 'critical',
                'actions': [
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod', parameters={'namespace': 'openshift-kube-apiserver', 'tail': 500}),
                    RemediationAction(ActionType.CHECK_EVENTS, 'namespace', parameters={'namespace': 'openshift-kube-apiserver'}),
                    RemediationAction(ActionType.CREATE_INCIDENT, 'incident', parameters={'severity': 'P1', 'type': 'platform'}),
                ],
                'escalation': 'page_platform_team'
            },
            {
                'condition': 'warning',
                'actions': [
                    RemediationAction(ActionType.COLLECT_LOGS, 'pod', parameters={'namespace': 'openshift-kube-apiserver'}),
                    RemediationAction(ActionType.SEND_ALERT, 'notification', parameters={'channel': 'platform'}),
                ]
            }
        ]
    },
}

print(f"✅ Defined remediation rules for {len(REMEDIATION_RULES)} metrics")
print(f"\nRules by category:")
for category in MetricCategory:
    count = sum(1 for m, r in REMEDIATION_RULES.items() if r['category'] == category)
    print(f"   {category.value}: {count} rules")



### 2. Anomaly to Rule Mapping

In [ ]:
def evaluate_threshold(metric_name: str, value: float) -> tuple[str, str]:
    """
    Evaluate if a metric value exceeds thresholds.
    
    Args:
        metric_name: Name of the metric
        value: Current metric value
    
    Returns:
        Tuple of (condition, severity) or (None, None) if normal
    """
    if metric_name not in METRIC_THRESHOLDS:
        return None, None
    
    threshold = METRIC_THRESHOLDS[metric_name]
    direction = threshold.get('direction', 'above')
    
    if direction == 'above':
        if value >= threshold['critical']:
            return 'critical', 'critical'
        elif value >= threshold['warning']:
            return 'warning', 'warning'
    else:  # below
        if value <= threshold['critical']:
            return 'critical', 'critical'
        elif value <= threshold['warning']:
            return 'warning', 'warning'
    
    return None, None


def map_anomaly_to_rules(metric_values: Dict[str, float]) -> List[Dict]:
    """
    Map detected anomalies to remediation rules for ALL 16 metrics.
    
    Args:
        metric_values: Dictionary of metric_name -> current_value
    
    Returns:
        List of triggered rules with actions
    """
    triggered_rules = []
    
    for metric_name, value in metric_values.items():
        if metric_name not in REMEDIATION_RULES:
            continue
        
        condition, severity = evaluate_threshold(metric_name, value)
        
        if condition is None:
            continue  # Value is normal
        
        rule_config = REMEDIATION_RULES[metric_name]
        
        # Find matching rule for this condition
        for rule in rule_config['rules']:
            if rule['condition'] == condition:
                triggered_rules.append({
                    'metric': metric_name,
                    'value': value,
                    'threshold': METRIC_THRESHOLDS[metric_name][condition],
                    'condition': condition,
                    'severity': rule_config['severity_map'].get(condition, 'medium'),
                    'description': rule_config['description'],
                    'category': rule_config['category'].value,
                    'actions': rule['actions'],
                    'escalation': rule.get('escalation'),
                })
                break
    
    # Sort by severity (critical first)
    severity_order = {'critical': 0, 'high': 1, 'medium': 2, 'low': 3}
    triggered_rules.sort(key=lambda x: severity_order.get(x['severity'], 4))
    
    return triggered_rules


# Test the mapping
test_metrics = {
    'node_memory_utilization': 92,    # Critical
    'pod_cpu_usage': 0.85,            # Warning
    'container_restart_count': 6,     # Critical
    'pods_pending': 5,                # Warning
    'apiserver_error_rate': 2,        # Warning
}

print("\n🧪 Testing anomaly mapping:")
print(f"   Input: {test_metrics}")
triggered = map_anomaly_to_rules(test_metrics)
print(f"   Triggered: {len(triggered)} rules")
for rule in triggered:
    print(f"   - [{rule['severity'].upper()}] {rule['metric']}: {rule['description']}")



In [ ]:
# =============================================================================
# CELL 7: KUBECTL COMMAND GENERATOR (new cell)
# =============================================================================

def generate_kubectl_command(action: RemediationAction, context: Dict) -> str:
    """
    Generate kubectl/oc command for a remediation action.
    
    Args:
        action: RemediationAction to execute
        context: Additional context (pod_name, namespace, etc.)
    
    Returns:
        kubectl/oc command string
    """
    namespace = context.get('namespace', action.namespace)
    target_name = context.get('target_name', action.target_name) or '*'
    
    commands = {
        ActionType.RESTART_POD: f"oc delete pod {target_name} -n {namespace}",
        
        ActionType.DELETE_POD: (
            f"oc delete pod {target_name} -n {namespace}"
            if action.parameters.get('field_selector') is None
            else f"oc delete pods --field-selector={action.parameters['field_selector']} -n {namespace}"
        ),
        
        ActionType.EVICT_POD: f"oc adm drain {context.get('node_name', 'NODE')} --pod-selector={action.parameters.get('selector', '')} --delete-emptydir-data --ignore-daemonsets",
        
        ActionType.SCALE_UP: f"oc scale deployment/{target_name} --replicas=$(( $(oc get deployment {target_name} -n {namespace} -o jsonpath='{{.spec.replicas}}') + {action.parameters.get('replicas_delta', 1)} )) -n {namespace}",
        
        ActionType.SCALE_DOWN: f"oc scale deployment/{target_name} --replicas=$(( $(oc get deployment {target_name} -n {namespace} -o jsonpath='{{.spec.replicas}}') - {abs(action.parameters.get('replicas_delta', 1))} )) -n {namespace}",
        
        ActionType.SCALE_HPA: f"oc patch hpa {target_name} -n {namespace} --type=merge -p '{{\"spec\":{{\"minReplicas\":$(( $(oc get hpa {target_name} -n {namespace} -o jsonpath='{{.spec.minReplicas}}') + {action.parameters.get('min_replicas_delta', 1)} ))}}}}'",
        
        ActionType.INCREASE_LIMITS: f"oc set resources deployment/{target_name} -n {namespace} --limits=cpu={action.parameters.get('cpu', '500m')},memory={action.parameters.get('memory', '512Mi')}",
        
        ActionType.EXPAND_PVC: f"oc patch pvc {target_name} -n {namespace} --type=merge -p '{{\"spec\":{{\"resources\":{{\"requests\":{{\"storage\":\"CALCULATED_SIZE\"}}}}}}}}'",
        
        ActionType.CLEANUP_STORAGE: f"# Manual: Review and delete files older than {action.parameters.get('older_than_days', 30)} days in PVC {target_name}",
        
        ActionType.CORDON_NODE: f"oc adm cordon {context.get('node_name', 'NODE')}",
        
        ActionType.DRAIN_NODE: f"oc adm drain {context.get('node_name', 'NODE')} --delete-emptydir-data --ignore-daemonsets --force",
        
        ActionType.UNCORDON_NODE: f"oc adm uncordon {context.get('node_name', 'NODE')}",
        
        ActionType.COLLECT_LOGS: (
            f"oc logs {target_name} -n {namespace} --tail={action.parameters.get('tail', 500)}"
            + (" --previous" if action.parameters.get('previous') else "")
            + (" --all-containers" if action.parameters.get('all_containers') else "")
        ),
        
        ActionType.DESCRIBE_RESOURCE: f"oc describe {action.target_type} {target_name} -n {namespace}",
        
        ActionType.CHECK_EVENTS: f"oc get events -n {namespace} --sort-by='.lastTimestamp' | tail -20",
        
        ActionType.SEND_ALERT: f"# Alert to {action.parameters.get('channel', 'default')}: {{message}}",
        
        ActionType.CREATE_INCIDENT: f"# Create {action.parameters.get('severity', 'P3')} incident: {{description}}",
        
        ActionType.MONITOR_ONLY: f"# Continue monitoring {target_name} for {action.parameters.get('duration', '5m')}",
    }
    
    return commands.get(action.action_type, f"# Unknown action: {action.action_type}")


# Test command generation
test_action = RemediationAction(ActionType.RESTART_POD, 'pod')
test_context = {'namespace': 'self-healing-platform', 'target_name': 'api-server-xyz'}
cmd = generate_kubectl_command(test_action, test_context)
print(f"\n🔧 Test command: {cmd}")



### 3. Execute Remediation Actions

In [ ]:
def execute_remediation(
    triggered_rules: List[Dict],
    namespace: str = 'self-healing-platform',
    dry_run: bool = True,
    auto_approve: bool = False
) -> List[Dict]:
    """
    Execute remediation actions for triggered rules.
    
    Args:
        triggered_rules: Output from map_anomaly_to_rules()
        namespace: Target namespace
        dry_run: If True, only generate commands without executing
        auto_approve: If True, skip approval for requires_approval actions
    
    Returns:
        List of execution results
    """
    results = []
    
    for rule in triggered_rules:
        logger.info(f"\n{'='*60}")
        logger.info(f"Processing: {rule['metric']} ({rule['severity'].upper()})")
        logger.info(f"Description: {rule['description']}")
        logger.info(f"Value: {rule['value']} (threshold: {rule['threshold']})")
        
        rule_results = {
            'metric': rule['metric'],
            'severity': rule['severity'],
            'actions': []
        }
        
        for action in rule['actions']:
            context = {
                'namespace': namespace,
                'target_name': None,  # Would come from incident data
            }
            
            cmd = generate_kubectl_command(action, context)
            
            action_result = {
                'action_type': action.action_type.value,
                'command': cmd,
                'requires_approval': action.requires_approval,
                'status': 'pending'
            }
            
            if action.requires_approval and not auto_approve:
                logger.warning(f"  ⚠️ REQUIRES APPROVAL: {action.action_type.value}")
                action_result['status'] = 'awaiting_approval'
            elif dry_run:
                logger.info(f"  [DRY RUN] Would execute: {cmd[:80]}...")
                action_result['status'] = 'dry_run'
            else:
                try:
                    logger.info(f"  Executing: {cmd[:80]}...")
                    # result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=60)
                    # action_result['stdout'] = result.stdout
                    # action_result['stderr'] = result.stderr
                    # action_result['status'] = 'success' if result.returncode == 0 else 'failed'
                    action_result['status'] = 'simulated'  # For safety
                except Exception as e:
                    logger.error(f"  Error: {e}")
                    action_result['status'] = 'error'
                    action_result['error'] = str(e)
            
            rule_results['actions'].append(action_result)
        
        # Check for escalation
        if rule.get('escalation'):
            logger.warning(f"  📢 Escalation: {rule['escalation']}")
            rule_results['escalation'] = rule['escalation']
        
        results.append(rule_results)
    
    return results


# Test execution
print("\n🚀 Testing remediation execution (dry run):")
execution_results = execute_remediation(triggered, dry_run=True)
print(f"\nExecuted {len(execution_results)} remediation rules")



### 4. Submit to Coordination Engine

In [ ]:
def submit_incident_to_engine(anomaly_data: Dict, triggered_rules: List[Dict]) -> Dict:
    """
    Submit incident to coordination engine with full context.
    
    Args:
        anomaly_data: Raw metric values that triggered anomalies
        triggered_rules: Output from map_anomaly_to_rules()
    
    Returns:
        Engine response
    """
    incident = {
        'timestamp': datetime.now().isoformat(),
        'type': 'multi_metric_anomaly',
        'metrics': anomaly_data,
        'triggered_rules': [
            {
                'metric': r['metric'],
                'severity': r['severity'],
                'description': r['description'],
                'category': r['category'],
                'action_count': len(r['actions']),
            }
            for r in triggered_rules
        ],
        'overall_severity': triggered_rules[0]['severity'] if triggered_rules else 'low',
        'status': 'pending'
    }
    
    try:
        if get_client:
            client = get_client()
            response = client.submit_incident(incident)
            logger.info(f"✅ Submitted incident to coordination engine")
            return response
        else:
            logger.warning("⚠️ Coordination engine client not available")
            return {'status': 'client_unavailable', 'incident': incident}
    except Exception as e:
        logger.error(f"❌ Error submitting incident: {e}")
        return {'error': str(e)}


# Test submission (commented out for safety)
# result = submit_incident_to_engine(test_metrics, triggered)
print("\n✅ Incident submission ready")



### 5. Track Remediation Outcomes

In [ ]:
import pandas as pd

remediation_log = []

def log_remediation(
    incident_id: str,
    metric: str,
    rule_severity: str,
    actions_executed: int,
    success: bool,
    duration_seconds: float,
    escalated: bool = False
):
    """
    Log remediation outcome with full details.
    """
    log_entry = {
        'timestamp': datetime.now().isoformat(),
        'incident_id': incident_id,
        'metric': metric,
        'severity': rule_severity,
        'actions_executed': actions_executed,
        'success': success,
        'duration_seconds': duration_seconds,
        'escalated': escalated
    }
    remediation_log.append(log_entry)
    logger.info(f"📝 Logged: {incident_id} - {metric} - {'✅' if success else '❌'}")


# Test logging
log_remediation('INC-001', 'node_memory_utilization', 'critical', 3, True, 15.5, escalated=True)
log_remediation('INC-002', 'container_restart_count', 'critical', 4, True, 8.2)
log_remediation('INC-003', 'pods_pending', 'warning', 2, True, 5.0)

# Save log
log_df = pd.DataFrame(remediation_log)
log_df.to_parquet(PROCESSED_DIR / 'remediation_log.parquet')
print(f"\n💾 Saved remediation log: {len(remediation_log)} entries")



## Validation Section

In [ ]:
# Verify outputs

# =============================================================================
# VALIDATION 
# =============================================================================

# Verify all 16 metrics have rules
assert len(REMEDIATION_RULES) == 16, f"Expected 16 rules, got {len(REMEDIATION_RULES)}"
assert len(METRIC_THRESHOLDS) == 16, f"Expected 16 thresholds, got {len(METRIC_THRESHOLDS)}"

# Verify all TARGET_METRICS have rules
for metric in TARGET_METRICS:
    assert metric in REMEDIATION_RULES, f"Missing rule for {metric}"
    assert metric in METRIC_THRESHOLDS, f"Missing threshold for {metric}"

# Verify outputs
assert (PROCESSED_DIR / 'remediation_log.parquet').exists(), "Remediation log not saved"

print("\n" + "="*60)
print("✅ ALL VALIDATIONS PASSED")
print("="*60)
print(f"\n📊 Summary:")
print(f"   Target Metrics: {len(TARGET_METRICS)}")
print(f"   Remediation Rules: {len(REMEDIATION_RULES)}")
print(f"   Metric Thresholds: {len(METRIC_THRESHOLDS)}")
print(f"   Action Types: {len(ActionType)}")
print(f"   Remediation Log Entries: {len(remediation_log)}")

print(f"\n📁 Output files:")
print(f"   - {PROCESSED_DIR / 'remediation_log.parquet'}")



In [ ]:


# =============================================================================
#  QUICK REFERENCE (easy lookup)
# =============================================================================

print("\n" + "="*60)
print("📋 QUICK REFERENCE: Metric → Actions")
print("="*60)

for metric in TARGET_METRICS:
    rule = REMEDIATION_RULES[metric]
    threshold = METRIC_THRESHOLDS[metric]
    critical_actions = [a.action_type.value for a in rule['rules'][0]['actions']]
    
    print(f"\n{metric}")
    print(f"  Category: {rule['category'].value}")
    print(f"  Thresholds: warning={threshold['warning']}, critical={threshold['critical']} ({threshold['unit']})")
    print(f"  Critical actions: {', '.join(critical_actions[:3])}{'...' if len(critical_actions) > 3 else ''}")


## Integration Section

This notebook integrates with:
- **Input**: Ensemble predictions from Phase 2
- **Output**: Remediation actions to coordination engine
- **Coordination Engine**: Executes remediation and tracks outcomes

## Next Steps

1. Review remediation rules
2. Proceed to `ai-driven-decision-making.ipynb`
3. Combine rule-based and AI-driven approaches
4. Deploy hybrid healing workflows

## References

- ADR-002: Hybrid Deterministic-AI Self-Healing Approach
- ADR-012: Notebook Architecture for End-to-End Workflows
- [Kubernetes API](https://kubernetes.io/docs/reference/generated/kubernetes-api/)